# Stock Analysis Notebook

## How to Use:

0. **Run the first code cell** to load all imports and utilities (optimized for performance)
1. **Change the stock**: Edit `user_input = 'AAPL'` to any stock symbol
2. **Run the selection cell** to load the stock data
3. **Explore the analysis sections** below (each can be run independently)

## Available Analysis Sections:

### Technical Analysis Dashboard
- **Main Chart**: Stock price with 20-day & 50-day moving averages and Bollinger Bands
- **Volume**: Trading volume in millions (color-coded by price direction)
- **RSI**: Relative Strength Index with overbought/oversold levels (70/30)
- **Volatility**: Annualized volatility percentage
- **Cumulative Returns**: Total returns from start of period
- **MACD**: Moving Average Convergence Divergence with signal line and histogram
- **Max Drawdown**: Peak-to-trough decline percentage over time
- **Stochastic Oscillator**: %K and %D lines with overbought/oversold levels (80/20)
- **Returns Distribution**: Histogram of daily returns distribution
- **On-Balance Volume**: Cumulative volume indicator (in millions)
- **Rolling Sharpe Ratio**: 60-day risk-adjusted performance metric

### Key Metrics Dashboard
Visual cards displaying: Current Price, Daily Change, 52-Week Range, Beta, P/E Ratio, Market Cap, Dividend Yield, and mini charts for Price, RSI, and Volatility trends

### Company Information
- **Competitors Analysis**: Top 10 competitors from CNN Business API
- **Institutional Holdings**: Top holders with shares and ownership percentages
- **Mutual Fund Holdings**: Top mutual fund positions
- **Insider Transactions**: Recent insider buying/selling activity
- **Company Officers**: Executive team roster with positions
- **News Feed**: Latest news articles with summaries and timestamps

### Financial Statements
- **Quarterly & Annual Income Statements**: Revenue and Net Income with Profit Margin trends
- **Quarterly & Annual Balance Sheets**: Total Assets and Total Debt with Debt-to-Asset ratios
- **Detailed Balance Sheet**: Full annual balance sheet with all line items
- **Insider Roster**: Current insider holdings and positions

### Analyst Coverage
- **Price Targets**: Historical stock price overlaid with analyst upgrade/downgrade distributions
- **Upgrade/Downgrade History**: Recent analyst rating changes

### Market Intelligence
- **CNN Fear & Greed Index**: Current sentiment score and rating
- **Historical Fear & Greed Trend**: Interactive chart showing sentiment evolution
- **Share Price Insights**: AI-generated analysis from CNN Business
- **Global Stock Market Map**: Interactive world map showing daily performance of major indices worldwide

### Options Analysis
- **Option Chain**: Calls and puts for nearest expiration
- **Put/Call Ratio**: Open interest ratio across expirations
- **Expected Move**: Implied volatility-based expected price movement
- **Open Interest by Strike**: Visual distribution of call and put OI
- **IV Term Structure**: Implied volatility across multiple expirations
- **Strike Table**: Comprehensive OI, volume, and IV data by strike

## Stock Symbol Examples:
- `user_input = 'AAPL'` for Apple
- `user_input = 'GOOGL'` for Google
- `user_input = 'TSLA'` for Tesla
- `user_input = 'NVDA'` for NVIDIA
- `user_input = 'MSFT'` for Microsoft
- `user_input = 'KO'` for Coca-Cola
- `user_input = 'SPY'` for S&P 500 ETF

## Table of Contents

- [Setup and Configuration](#setup-and-configuration)
- [Stock Selection](#stock-selection)  
- [Technical Analysis Dashboard](#technical-analysis-dashboard)
- [Company Information](#company-information)
- [Analyst Price Targets](#analyst-price-targets)
- [Market Intelligence](#market-intelligence)

[Back to Top](#table-of-contents)

## Setup and Configuration

In [ ]:
# Core imports
import yfinance as yf
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.gridspec import GridSpec
from statsmodels.tsa.stattools import coint
from statsmodels.tsa.vector_ar.vecm import coint_johansen
import scipy.stats as stats
from typing import Dict, List, Optional
from pprint import pprint
import warnings
warnings.filterwarnings('ignore')
import functions
import importlib
importlib.reload(functions)

# Additional imports for display and web requests
from IPython.display import Markdown, display
import requests
from urllib3.util.retry import Retry
from requests.adapters import HTTPAdapter
import time
import json

# Display and utility imports
from IPython.display import HTML, display, Markdown
from io import BytesIO
import base64

# Import utility functions
from functions import (
    get_ticker, 
    fetch_stock_data, 
    setup_date_formatting, 
    humanize_number, 
    safe_float, 
    get_current_price_and_date,
    display_structured_data_as_markdown
)

In [ ]:
# Matplotlib Style Configuration
# Change this line to use different graph styles

plt.style.use('default')            # Clean white background

# Custom style settings (optional)
plt.rcParams.update({
    'figure.figsize': (12, 8),      # Default figure size
    'font.size': 10,                # Default font size
    'axes.labelsize': 12,           # Axis label size
    'axes.titlesize': 14,           # Title size
    'xtick.labelsize': 10,          # X-tick label size
    'ytick.labelsize': 10,          # Y-tick label size
    'legend.fontsize': 10,          # Legend font size
    'figure.dpi': 200,              # Resolution
})

# List available styles
#for i, style in enumerate(sorted(plt.style.available), 1):
    #print(f"  {i:2d}. {style}")

In [ ]:
# Utility function for making API requests to CNN Business
def make_cnn_api_request(url, max_retries=3, backoff_factor=1.0, timeout=10):
    """
    Make an API request to CNN Business with retry logic.
    
    Args:
        url: The API endpoint URL
        max_retries: Number of retry attempts
        backoff_factor: Factor for exponential backoff
        timeout: Request timeout in seconds
    
    Returns:
        Response object or None if all retries failed
    """
    session = requests.Session()
    retry = Retry(
        total=max_retries,
        backoff_factor=backoff_factor,
        status_forcelist=[500, 502, 503, 504]
    )
    adapter = HTTPAdapter(max_retries=retry)
    session.mount('http://', adapter)
    session.mount('https://', adapter)
    
    try:
        response = session.get(url, timeout=timeout)
        return response
    except requests.exceptions.RequestException as e:
        print(f"Request failed: {e}")
        return None

## Stock Selection

**How to change stocks:**
1. In the cell below, change `user_input = 'AAPL'` to any stock symbol you want
2. Examples: `'GOOGL'`, `'MSFT'`, `'TSLA'`, `'NVDA'`, `'AMZN'`, etc.
3. Run the cell above, then run the dashboard cell below

In [ ]:
selected_stock = 'TSLA'  # Change this to any stock symbol (e.g., 'AAPL', 'GOOGL', 'MSFT', etc.)
period = '1y'  # Change this to desired period (e.g., '1d', '1wk', '6mo', '1y', '2y', 'max' etc.) - x-axis labels will auto-adjust

In [ ]:
# STOCK SELECTION
# Change the stock symbol below to any stock you want to analyze
print(f"Fetching data for {selected_stock}...")

# Fetch stock data
data = fetch_stock_data(selected_stock, period)

if not data.empty:
    # Calculate technical indicators using utility function
    from functions import calculate_technical_indicators
    data = calculate_technical_indicators(data)
    
    # Calculate additional metrics
    data['Daily_Return'] = data['Close'].pct_change()
    data['Volatility'] = data['Daily_Return'].rolling(window=20).std() * np.sqrt(252)
    data['Cumulative_Return'] = (1 + data['Daily_Return']).cumprod() - 1
    
    ticker = get_ticker(selected_stock)  # Ensure ticker is set
    print(f"Successfully loaded {selected_stock}")
else:
    print(f"No data found for {selected_stock}")

## Technical Analysis Dashboard

In [ ]:
# Create dashboard visualization
print(f"Creating dashboard for {selected_stock}...")
if not isinstance(data,pd.DataFrame)or data.empty:print("Error: No valid stock data. Run STOCK SELECTION cell first.");raise SystemExit
pn={'1d':'1 Day','5d':'5 Days','1mo':'1 Month','3mo':'3 Months','6mo':'6 Months','1y':'1 Year','2y':'2 Years','5y':'5 Years','10y':'10 Years','ytd':'Year to Date','max':'Maximum'}
prd=pn.get(period,period);cp=data['Close'];ix,li=data.index,data.index[-1];rs20=cp.rolling(20).std();dr=data['Daily_Return']
ub,lb=data['MA_20']+(rs20*2),data['MA_20']-(rs20*2);e12,e26=cp.ewm(span=12).mean(),cp.ewm(span=26).mean();macd=e12-e26;sig=macd.ewm(span=9).mean();hist=macd-sig
cum=(1+dr).cumprod();dd=((cum-cum.cummax())/cum.cummax())*100;lhr=data['High'].rolling(14).max()-data['Low'].rolling(14).min()
kp=100*((cp-data['Low'].rolling(14).min())/lhr);dp=kp.rolling(3).mean();obv=(np.sign(cp.diff())*data['Volume']).fillna(0).cumsum();rsh=(dr.rolling(60).mean()/dr.rolling(60).std())*np.sqrt(252)

def an(a,v,x,c='#114f7a'):
    if not np.isnan(v):a.annotate(f'{v:.1f}',xy=(x,v),xytext=(5,5),textcoords='offset points',fontsize=8,fontweight='bold',bbox=dict(boxstyle='round,pad=0.2',facecolor=c,alpha=0.2))

def st(a,t,y='',l=True,tk=8):a.set_title(t,fontweight='bold',fontsize=11);a.set_ylabel(y,fontsize=9)if y else None;a.tick_params(axis='both',labelsize=tk);a.legend(fontsize=7)if l else None

fig=plt.figure(figsize=(14,16));gs=GridSpec(6,3,height_ratios=[4,1,1.5,1.5,1.5,2],hspace=0.4,wspace=0.3)
am=fig.add_subplot(gs[0,:]);am.plot(ix,cp,color="#114f7a",label=f'{selected_stock} Close',linewidth=2.5)
am.plot(ix,data['MA_20'],color="#114e7ae0",alpha=0.8,linestyle='--',label='20-day MA',linewidth=1.5)
am.plot(ix,data['MA_50'],color="#114e7ac8",alpha=0.8,linestyle=':',label='50-day MA',linewidth=1.5)
am.fill_between(ix,lb,ub,color='gray',alpha=0.2,label='Bollinger Bands')
am.set_title(f'{selected_stock} Stock Price with Moving Averages & Bollinger Bands ({prd})',fontsize=16,fontweight='bold',pad=20)
am.set_ylabel('Price ($)',fontsize=12);am.legend(fontsize=10,loc='upper left');am.grid(True,alpha=0.3);am.tick_params(labelbottom=False)
am.annotate(f'${cp.iloc[-1]:.2f}',xy=(li,cp.iloc[-1]),xytext=(10,10),textcoords='offset points',bbox=dict(boxstyle='round,pad=0.3',facecolor='yellow',alpha=0.7),fontsize=10,fontweight='bold')

av=fig.add_subplot(gs[1,:],sharex=am);vd=data['Volume']/1e6;cols=np.where(cp.diff()>0,'#00ff00','#ff0000');cols[0]='#d62728'
av.bar(ix,vd,color=cols,alpha=0.6,width=1);av.set_ylabel('Volume\n(Millions)',fontsize=10);av.grid(True,alpha=0.3);av.tick_params(labelsize=9)

for pos,y,t,yl,col,ex in[(gs[2,0],data['RSI'],'RSI','RSI','#114f7a',lambda a:(a.axhline(70,color='red',linestyle='--',alpha=0.7,label='Overbought (70)'),a.axhline(30,color='green',linestyle='--',alpha=0.7,label='Oversold (30)'),a.set_ylim(0,100),an(a,data['RSI'].iloc[-1],li,'#9467bd'))),(gs[2,1],data['Volatility']*100,'Volatility','Volatility (%)','#114f7a',lambda a:an(a,(data['Volatility']*100).iloc[-1],li)),(gs[2,2],data['Cumulative_Return']*100,'Cumulative Returns','Cumulative Returns (%)','#114f7a',lambda a:an(a,(data['Cumulative_Return']*100).iloc[-1],li))]:
    a=fig.add_subplot(pos);a.plot(ix,y,color=col,linewidth=2,alpha=0.8);st(a,t,yl,False);ex(a)

amd=fig.add_subplot(gs[3,:2]);amd.plot(ix,macd,label='MACD',color='blue',linewidth=1.5);amd.plot(ix,sig,label='Signal',color='red',linewidth=1)
amd.bar(ix,hist,color='gray',alpha=0.5,width=1);amd.axhline(y=0,color='black',linestyle='--',alpha=0.5);st(amd,'MACD','',True,8)

add=fig.add_subplot(gs[3,2]);add.fill_between(ix,dd,0,color='red',alpha=0.3);st(add,'Max Drawdown (%)','Drawdown (%)',False);an(add,dd.iloc[-1],li,'red')

ast=fig.add_subplot(gs[4,0]);ast.plot(ix,kp,label='%K',color='blue');ast.plot(ix,dp,label='%D',color='red')
ast.axhline(80,color='red',linestyle='--',alpha=0.7,label='Overbought');ast.axhline(20,color='green',linestyle='--',alpha=0.7,label='Oversold');st(ast,'Stochastic Oscillator','')

ah=fig.add_subplot(gs[4,1]);ah.hist(dr.dropna()*100,bins=50,alpha=0.7,color='purple',edgecolor='black')
ah.set_title('Daily Returns Distribution',fontweight='bold',fontsize=11);ah.set_xlabel('Return (%)');ah.axvline(x=0,color='red',linestyle='--');ah.tick_params(axis='both',labelsize=8)

ao=fig.add_subplot(gs[4,2]);ao.plot(ix,obv/1e6,color='orange',linewidth=2);st(ao,'On-Balance Volume (Millions)','',False);ao.grid(True,alpha=0.3)

asr=fig.add_subplot(gs[5,:]);asr.plot(ix,rsh,color='green',linewidth=2);asr.set_title('Rolling Sharpe Ratio (60-day)',fontweight='bold',fontsize=10)
asr.axhline(y=0,color='black',linestyle='--',alpha=0.5);asr.grid(True,alpha=0.3);asr.tick_params(axis='both',labelsize=8)

setup_date_formatting(asr,period);plt.tight_layout();plt.subplots_adjust(bottom=0.15);plt.show()

In [ ]:
# Display key metrics with visual cards and mini charts
from IPython.display import HTML, display
from io import BytesIO
import base64, matplotlib.pyplot as plt
from functions import humanize_number, safe_float, get_current_price_and_date

def ch(v,c,cl='#3b82f6'):
    if not v or all(x is None for x in v):return None
    v=[None]*(4-len(v[-4:]))+v[-4:];f,a=plt.subplots(figsize=(1.2,.7));f.patch.set_alpha(0);a.set_facecolor('none');b=a.bar(range(4),[x or 0 for x in v],color=cl,width=.7)
    for x in b:x.set_capstyle('round');x.set_joinstyle('round')
    a.axis('off');plt.tight_layout(pad=0);u=BytesIO();plt.savefig(u,format='png',dpi=50,bbox_inches='tight',transparent=True);plt.close(f);u.seek(0);return f'data:image/png;base64,{base64.b64encode(u.read()).decode()}'

def cd(n,v,c=None,h=None,s=None):
    ch=f'<div style="color: {"#10b981"if c>0else"#ef4444"if c<0else"#6b7280"}; font-size: 12px; margin-top: 4px;">{"▲"if c>0else"▼"if c<0else"●"} {abs(c):.1f}%</div>'if c is not None else''
    sb=f'<div style="color: #9ca3af; font-size: 11px; margin-top: 2px;">{s}</div>'if s else''
    st='background: white; border: 1px solid #e5e7eb; border-radius: 8px; padding: 16px; min-width: 180px; box-shadow: 0 1px 3px rgba(0,0,0,0.1);'
    ct=f'<div style="color: #6b7280; font-size: 13px; font-weight: 500; margin-bottom: 8px;">{n}</div><div style="color: #111827; font-size: 24px; font-weight: 700;">{v}</div>{sb}{ch}'
    return f'<div style="{st} display: flex; justify-content: space-between; align-items: center; gap: 12px;"><div style="flex: 1;">{ct}</div><div style="flex-shrink: 0;"><img src="{h}" style="width: 70px; height: 40px;"></div></div>'if h else f'<div style="{st}">{ct}</div>'

def pc(n,c):
    if c is None:return cd(n,'N/A')
    cl='#10b981'if c>0else'#ef4444'if c<0else'#6b7280';sy='▲'if c>0else'▼'if c<0else'●';v=f'{sy} {abs(c):.1f}%'if any(k in n for k in['%','Return','Volatility','Drawdown','VaR'])else f'{sy} {abs(c):.2f}'
    return f'<div style="background: white; border: 1px solid #e5e7eb; border-radius: 8px; padding: 16px; min-width: 180px; box-shadow: 0 1px 3px rgba(0,0,0,0.1);"><div style="color: #6b7280; font-size: 13px; font-weight: 500; margin-bottom: 8px;">{n}</div><div style="color: {cl}; font-size: 24px; font-weight: 700;">{v}</div></div>'

if 'selected_stock'not in globals()or not selected_stock:display(HTML('<div style="color: #dc2626; font-weight: 600;">No stock selected.</div>'))
else:
    t=yf.Ticker(selected_stock);rh=t.history(period='5d')
    if'data'not in globals()or data is None or(isinstance(data,pd.DataFrame)and data.empty):
        data=t.history(period='2y')
        if isinstance(data,pd.DataFrame)and not data.empty:from functions import calculate_technical_indicators;data=calculate_technical_indicators(data)
    i=getattr(t,'info',{})or{};qe=getattr(t,'earnings_dates',None);qeps=qe['EPS'].dropna().tail(4).tolist()if qe is not None and not qe.empty and'EPS'in qe.columns else[]
    qv={}
    if isinstance(data,pd.DataFrame)and not data.empty and isinstance(data.index,pd.DatetimeIndex):qd=data.resample('Q').last();qd=qd.iloc[-5:-1]if len(qd)>=5else qd;qv={c:qd[c].tolist()for c in['Close','RSI','Volume']if c in qd.columns}
    hp,hr,hv=qv.get('Close',[]),qv.get('RSI',[]),qv.get('Volume',[]);cp,ld=get_current_price_and_date(rh,data,i);pc1=None
    if isinstance(rh,pd.DataFrame)and len(rh)>=2:p0,p1=safe_float(rh['Close'].iloc[-2]),safe_float(rh['Close'].iloc[-1]);pc1=(p1-p0)/p0*100 if p0 else None
    dy=i.get('dividendYield');dyp=(dy*100 if dy and dy<0.5else dy if dy and dy<20else None)if dy else None
    m={k:i.get(v)for k,v in[('market_cap','marketCap'),('pe_ratio','trailingPE'),('eps','trailingEps'),('pb_ratio','priceToBook'),('week_52_high','fiftyTwoWeekHigh'),('week_52_low','fiftyTwoWeekLow'),('volume','volume'),('beta','beta'),('ebitda','ebitda'),('book_value','bookValue'),('shares_outstanding','sharesOutstanding')]}
    m.update({'dividend_yield':dyp,'company_name':i.get('longName')or i.get('shortName')or selected_stock,'sector':i.get('sector','N/A'),'industry':i.get('industry','N/A')})
    tech={}
    if isinstance(data,pd.DataFrame):
        for ind,col in[('rsi','RSI'),('macd','MACD'),('stochastic_k','%K'),('volume','Volume')]:
            if col in data.columns and not(s:=data[col].dropna()).empty:tech[ind]=safe_float(s.iloc[-1])
        if all(c in data.columns for c in['Close','Upper_Band','Lower_Band']):cv,u,l=safe_float(data['Close'].iloc[-1]),safe_float(data['Upper_Band'].iloc[-1]),safe_float(data['Lower_Band'].iloc[-1]);tech['bollinger_b']=(cv-l)/(u-l)if cv and u and l and u!=l else None
    pf={}
    if isinstance(data,pd.DataFrame)and not data.empty and'Close'in data.columns:
        cl=data['Close']
        for n,d in[('30d',30),('90d',90),('6m',126),('1y',252)]:pf[n]=((cl.iloc[-1]-cl.iloc[-d])/cl.iloc[-d])*100 if len(data)>=d else((cl.iloc[-1]-cl.iloc[0])/cl.iloc[0])*100 if n=='1y'and len(data)>126else None
    rk={k:None for k in['sharpe_ratio','sortino_ratio','calmar_ratio','annualized_volatility','annualized_return','max_drawdown','var_95']}
    if isinstance(data,pd.DataFrame)and not data.empty and'Close'in data.columns and len(data)>=30:
        rt=data['Close'].pct_change().dropna()
        if not rt.empty:tr=(data['Close'].iloc[-1]/data['Close'].iloc[0])-1;yr=len(data)/252;rk['annualized_return']=((1+tr)**(1/yr)-1)*100 if yr>0else None;rk['annualized_volatility']=rt.std()*np.sqrt(252)*100;rk['sharpe_ratio']=(rk['annualized_return']or 0)/rk['annualized_volatility']if rk['annualized_volatility']and rk['annualized_volatility']>0else None;cm=(1+rt).cumprod();dd=(cm-cm.expanding().max())/cm.expanding().max();rk['max_drawdown']=dd.min()*100;dr=rt[rt<0];rk['sortino_ratio']=(rk['annualized_return']or 0)/((ds:=dr.std()*np.sqrt(252))*100)if len(dr)>0and(ds:=dr.std()*np.sqrt(252))>0else None;rk['calmar_ratio']=rk['annualized_return']/abs(rk['max_drawdown'])if rk['max_drawdown']and rk['max_drawdown']<0and rk['annualized_return']else None;rk['var_95']=abs(rt.quantile(0.05))*100
    chg={}
    if m['market_cap']and hp and cp and cp!=0:m1y=(data['Close'].iloc[-250]if isinstance(data,pd.DataFrame)and not data.empty and len(data)>=250else hp[0])*(m['market_cap']/cp);chg['market_cap']=((m['market_cap']-m1y)/m1y)*100 if m1y and m1y!=0else None
    if m['eps']and qeps and qeps[0]!=0:chg['eps']=((m['eps']-qeps[0])/qeps[0])*100
    cht={}
    if hp and cp:cht['price']=ch(hp,cp)
    if m['market_cap']and hp and cp and cp!=0:cht['market_cap']=ch([p*(m['market_cap']/cp)for p in hp],m['market_cap'])
    if m['pe_ratio']and qeps and(qpe:=[cp/e for e in qeps if e and e!=0]):cht['pe']=ch(qpe,m['pe_ratio'])
    if m['eps']and qeps:cht['eps']=ch(qeps,m['eps'])
    if m['pb_ratio']and hp and m['book_value']and m['book_value']!=0:cht['pb']=ch([p/m['book_value']for p in hp],m['pb_ratio'])
    if tech.get('rsi')and hr:cht['rsi']=ch(hr,tech['rsi'])
    if tech.get('volume')and hv:cht['volume']=ch(hv,tech['volume'])
    h=[f'''<div style="font-family: -apple-system, BlinkMacSystemFont, 'Segoe UI', Roboto, sans-serif; padding: 20px;"><h2 style="color: #111827; margin-bottom: 8px;">{m['company_name']} ({selected_stock})</h2><div style="color: #6b7280; font-size: 14px; margin-bottom: 24px;">{m['sector']} • {m['industry']}</div><div style="display: grid; grid-template-columns: repeat(auto-fill, minmax(180px, 1fr)); gap: 16px; margin-bottom: 24px;">''']
    for n,v,c,ch,s in[('Current Price',f'${cp:.2f}'if cp else'N/A',pc1,cht.get('price'),None),('Market Cap',f'${humanize_number(m["market_cap"])}'if m['market_cap']else'N/A',chg.get('market_cap'),cht.get('market_cap'),None),('Shares Outstanding',humanize_number(m['shares_outstanding'])if m['shares_outstanding']else'N/A',None,None,None),('P/E Ratio',f'{m["pe_ratio"]:.2f}'if m['pe_ratio']else'N/A',None,cht.get('pe'),None),('EPS',f'${m["eps"]:.2f}'if m['eps']else'N/A',chg.get('eps'),cht.get('eps'),None),('EBITDA',f'${humanize_number(m["ebitda"])}'if m['ebitda']else'N/A',None,None,None),('Dividend Yield',f'{m["dividend_yield"]:.2f}%'if m['dividend_yield']else'N/A',None,None,None),('P/B Ratio',f'{m["pb_ratio"]:.2f}'if m['pb_ratio']else'N/A',None,cht.get('pb'),None),('52W High',f'${m["week_52_high"]:.2f}'if m['week_52_high']else'N/A',None,None,None),('52W Low',f'${m["week_52_low"]:.2f}'if m['week_52_low']else'N/A',None,None,None),('Volume',humanize_number(m['volume']or tech.get('volume'))if(m['volume']or tech.get('volume'))else'N/A',None,cht.get('volume'),None),('RSI',f'{tech["rsi"]:.1f}'if tech.get('rsi')else'N/A',None,cht.get('rsi'),None),('MACD',f'{tech["macd"]:.3f}'if tech.get('macd')is not None else'N/A',None,None,None),('Bollinger %B',f'{tech["bollinger_b"]:.2f}'if tech.get('bollinger_b')is not None else'N/A',None,None,None),('Stochastic %K',f'{tech["stochastic_k"]:.1f}'if tech.get('stochastic_k')is not None else'N/A',None,None,None),('Beta',f'{m["beta"]:.2f}'if m['beta']else'N/A',None,None,'(vs S&P 500)')]:h.append(cd(n,v,c,ch,s))
    h.append('</div><h3 style="color: #111827; margin-top: 24px; margin-bottom: 16px;">Risk Metrics</h3><div style="display: grid; grid-template-columns: repeat(auto-fill, minmax(180px, 1fr)); gap: 16px;">')
    for n,k in[('Sharpe Ratio','sharpe_ratio'),('Sortino Ratio','sortino_ratio'),('Calmar Ratio','calmar_ratio'),('Annualized Volatility','annualized_volatility'),('Annualized Return','annualized_return'),('Max Drawdown','max_drawdown'),('VaR (95%)','var_95')]:h.append(pc(n,rk.get(k)))
    h.append('</div><h3 style="color: #111827; margin-top: 24px; margin-bottom: 16px;">Performance</h3><div style="display: grid; grid-template-columns: repeat(auto-fill, minmax(180px, 1fr)); gap: 16px;">')
    for n,k in[('30-Day Return','30d'),('90-Day Return','90d'),('6-Month Return','6m'),('1-Year Return','1y')]:h.append(pc(n,pf.get(k)))
    h.append('</div></div>');display(HTML(''.join(h)))

[Back to Top](#table-of-contents)

## Company infomations

In [ ]:
# Render latest news articles
try:
    news = getattr(ticker, 'news', None)
    if not news:
        display(Markdown("**No news available.**"))
    else:
        from datetime import datetime
        md = [f"### News for {selected_stock} — {len(news)} articles found", '']
        
        for i, article in enumerate(news, 1):
            try:
                content = article.get('content', {}) if isinstance(article, dict) else {}
                provider = (content.get('provider', {}) if isinstance(content, dict) else {}).get('displayName', 'Unknown')
                summary = content.get('summary') or content.get('description') or content.get('title') or 'No summary'
                pub_date = content.get('pubDate') or content.get('displayTime') or article.get('pubDate') or article.get('date') or 'Unknown date'
                
                if pub_date and pub_date != 'Unknown date':
                    try:
                        pub_date = datetime.fromisoformat(str(pub_date).replace('Z', '+00:00')).strftime('%Y-%m-%d %H:%M:%S')
                    except:
                        pub_date = str(pub_date)
                
                summary_short = (summary[:150] + '...') if isinstance(summary, str) and len(summary) > 150 else summary
                md.append(f"{i}. **{provider}** — {summary_short} ({pub_date})")
            except Exception as e:
                md.append(f"{i}. Error: {e}")
        
        display(Markdown('\n\n'.join(md)))
except Exception as e:
    display(Markdown(f"**Error:** {e}"))

In [ ]:
# Company Officers, Institutional Holdings, and Mutual Fund Holdings
from IPython.display import Markdown, display

# --- Company Officers ---
try:
    ticker_obj = globals().get('ticker') or yf.Ticker(selected_stock)
    officers = ticker_obj.info.get('companyOfficers')
    display_structured_data_as_markdown(officers, "Company Officers", selected_stock)
except Exception as e:
    display(Markdown(f"**Error displaying company officers:** {e}"))

# --- Institutional Holdings ---
try:
    info = ticker_obj.info or {}
    holders = ticker_obj.institutional_holders
    if holders is None or holders.empty:
        raise ValueError()
    
    shares_out = info.get('sharesOutstanding')
    held_pct = info.get('heldPercentInstitutions')
    
    lines = [f"### Top Institutional Holdings for {selected_stock}",
             f"**Percent held by Institutions:** {held_pct*100:.2f}%" if held_pct else "**Percent held by Institutions:** N/A", ""]
    
    for i, row in enumerate(holders.head(10).itertuples(index=False), 1):
        try:
            holder = getattr(row, 'Holder', list(row)[0])
            shares = float(getattr(row, 'Shares', list(row)[1] if len(list(row)) > 1 else 0) or 0)
            shares_str = f"{humanize_number(shares)} shares{f' ({(shares/shares_out)*100:.2f}%)' if shares and shares_out and shares_out > 0 else ''}" if shares else 'N/A'
            lines.append(f"{i}. **{holder}** — {shares_str}")
        except:
            lines.append(f"{i}. **Unknown** — N/A")
    
    display(Markdown('\n\n'.join(lines)))
except:
    display(Markdown(f"**Institutional Holdings:** Not available for {selected_stock}"))

# --- Mutual Fund Holdings ---
try:
    shares_outstanding = info.get('sharesOutstanding')
    mutualfund_holders = ticker_obj.mutualfund_holders
    
    if mutualfund_holders is not None and not mutualfund_holders.empty:
        md_lines = [f"### Top Mutual Fund Holdings for {selected_stock}", ""]
        
        for i, row in enumerate(mutualfund_holders.head(10).itertuples(index=False), 1):
            try:
                holder = getattr(row, 'Holder', None) or list(row)[0]
            except:
                holder = 'Unknown'
            try:
                shares = getattr(row, 'Shares', None) or (list(row)[1] if len(list(row)) > 1 else 0)
            except:
                shares = 0

            shares_str = 'N/A'
            if shares and shares != 0:
                try:
                    shares_val = float(shares)
                    shares_str = f"{humanize_number(shares_val)} shares"
                    if shares_outstanding and shares_outstanding > 0:
                        pct = (shares_val / shares_outstanding) * 100
                        shares_str += f" ({pct:.2f}%)"
                except:
                    shares_str = str(shares)

            md_lines.append(f"{i}. **{holder}** — {shares_str}")

        display(Markdown('\n\n'.join(md_lines)))
    else:
        display(Markdown(f"**Mutual Fund Holdings:** Not available for {selected_stock}"))
except Exception as e:
    display(Markdown(f"**Error fetching mutual fund holdings data:** {e}"))

In [ ]:
# Fetch and display competitors from CNN Business API
from functions import make_cnn_api_request

try:
    resp = make_cnn_api_request(f"https://production.dataviz.cnn.io/quote/competitors/{selected_stock}/10")
    
    if resp is None:
        raise ValueError("No response received (network error)")
    if resp.status_code != 200:
        raise ValueError(f"Status {resp.status_code}: {resp.text[:1000]}")
    
    data = resp.json()
    items = data.get("competitors", data) if isinstance(data, dict) else data if isinstance(data, list) else []
    
    norm = lambda s: str(s or "").upper().strip().split("-")[0].split(".")[0] if s else ""
    target = norm(selected_stock)
    
    competitors_list = [
        {"symbol": (sym := norm(c.get("symbol") or c.get("ticker") or c.get("ticker_symbol") or "")) or "N/A",
         "name": c.get("name") or c.get("companyName") or "Unknown"}
        for c in items if norm(c.get("symbol") or c.get("ticker") or c.get("ticker_symbol") or "") != target
    ][:10]
    
    if competitors_list:
        md = f"### Top {len(competitors_list)} competitors for {selected_stock}\n\n" + \
             "\n".join(f"{i}. **{c['symbol']}** — {c['name']}" for i, c in enumerate(competitors_list, 1))
    else:
        md = f"**No competitors found**\n\n{json.dumps(items, indent=2)[:4000]}"
    
    display(Markdown(md))
except Exception as e:
    display(Markdown(f"**Error fetching competitors:** {e}"))

### Insider information

In [ ]:
# Insider Transactions
try:
    ins = yf.Ticker(selected_stock).insider_transactions
    if ins is None or ins.empty:
        raise ValueError("No data")
    
    df_ins = ins.copy()
    date_col = next((c for c in ['Start Date', 'FilingDate', 'Date', 'TransactionDate'] if c in df_ins.columns), None)
    if date_col:
        df_ins.sort_values(by=date_col, ascending=False, inplace=True)
    
    lines = [f"### Insider Transactions for {selected_stock}\n"]
    for _, row in df_ins.head(10).iterrows():
        date_str = f"{pd.to_datetime(row.get(date_col, '')).date()}" if pd.notna(row.get(date_col)) else 'Unknown date'
        title = row.get('Title', row.get('Position', '')).strip()
        who = f"**{title}**, {row.get('Insider', 'Unknown')}" if title else row.get('Insider', 'Unknown')
        action = row.get('Text', 'Transaction').strip()
        shares_h = humanize_number(row.get('Shares', row.get('SharesTransacted', row.get('SharesChanged', None))))
        value_h = humanize_number(row.get('Value', row.get('ValueTransacted', None)))
        lines.append(f"- {date_str}: {who} **{action}** ({shares_h} shares) for an overall value of **${value_h}**")
    
    display(Markdown("\n".join(lines)))
except Exception as e:
    print(f'No insider transactions available for {selected_stock}')

In [ ]:
# Format insider_transactions with humanize_number
insider_trans = ticker.insider_transactions.head(10)
numeric_cols = insider_trans.select_dtypes(include=[np.number]).columns
insider_trans[numeric_cols] = insider_trans[numeric_cols].applymap(humanize_number)
insider_trans

In [ ]:
roster_holder = ticker.insider_roster_holders

# Select only numeric columns and apply humanize_number to them
numeric_cols = roster_holder.select_dtypes(include=[np.number]).columns
roster_holder[numeric_cols] = roster_holder[numeric_cols].applymap(humanize_number)

# Display the DataFrame
with pd.option_context('display.max_rows', None, 'display.max_columns', None, 'display.width', None, 'display.max_colwidth', None):
    display(roster_holder)

[Back to Top](#table-of-contents)

## Financial Statements

In [ ]:
# Quarterly and Yearly Income Statement and Balance Sheet with values in Billions
fig, axes = plt.subplots(2, 2, figsize=(18, 10))

for row, (inc, bal, period, fmt) in enumerate([
    (ticker.quarterly_income_stmt, ticker.quarterly_balance_sheet, 'Quarterly', 
     lambda d: [f"Q{((x.month-1)//3)+1} {x.year}" for x in d]),
    (ticker.income_stmt, ticker.balance_sheet, 'Annual', lambda d: [x.year for x in d])
]):
    for col, (metrics, title_mid, ratio_label, ylim) in enumerate([
        ({'Revenue': inc.loc["Total Revenue"]/1e9, 'Net Income': inc.loc["Net Income"]/1e9},
         'Total Revenue and Net Income', 'Profit Margin (%)', None),
        ({'Total Assets': bal.loc["Total Assets"]/1e9, 'Total Debt': bal.loc["Total Debt"]/1e9},
         'Total Assets and Total Debt', 'Debt-to-Asset Ratio (%)', 100)
    ]):
        df = pd.DataFrame(metrics).sort_index().tail(4)
        ax = axes[row, col]
        df.plot(kind='bar', color=["#000000", "#2a5fff"], width=0.8, ax=ax)
        ax.set_title(f'{period} {title_mid} (in Billions) with {ratio_label}')
        ax.set_ylabel('Amount (Billions USD)')
        ax.set_xticks(range(len(df)))
        ax.set_xticklabels(fmt(df.index), rotation=0)
        ax.grid(axis='y', linestyle='--', alpha=0.7)
        
        ratio = (df.iloc[:, 1] / df.iloc[:, 0]) * 100
        ax_twin = ax.twinx()
        ax_twin.plot(range(len(ratio)), ratio.values, color='#fb950f', marker='o', linewidth=2, label=ratio_label)
        ax_twin.set_ylabel(ratio_label, color="#000000")
        ax_twin.tick_params(axis='y', labelcolor="#000000")
        ax_twin.set_ylim(bottom=0, top=ylim)
        
        lines1, labels1 = ax.get_legend_handles_labels()
        lines2, labels2 = ax_twin.get_legend_handles_labels()
        ax.legend(lines1 + lines2, labels1 + labels2, title='Metrics', bbox_to_anchor=(1.05, 1), loc='upper left')

plt.tight_layout()
plt.show()

In [ ]:
# Display annual balance sheet with all rows visible
bs = ticker.balance_sheet
bs = bs.applymap(humanize_number)
with pd.option_context('display.max_rows', None, 'display.max_columns', None, 'display.width', None, 'display.max_colwidth', None):
    display(bs)

## Analyst Coverage

In [ ]:
# Take a closer look at this - Analyst upgrades and downgrades data
ticker.upgrades_downgrades.head(10)

### Analyst Price Targets Visualization

This chart combines the stock price history with analyst price target distributions from upgrades/downgrades data.

In [ ]:
# Plot Recommendations Summary
try:
    rec = ticker.recommendations_summary
except (NameError, AttributeError):
    display(Markdown('**`ticker` not defined. Run STOCK SELECTION cell first.**'))
    rec = None

if rec is None or (isinstance(rec, pd.DataFrame) and rec.empty):
    display(Markdown('**No recommendations_summary available.**'))
else:
    try:
        if isinstance(rec, pd.DataFrame):
            latest = rec.iloc[-1]
            col_map = {'strongBuy': 'Strong Buy', 'strong_buy': 'Strong Buy', 'Strong Buy': 'Strong Buy',
                      'buy': 'Buy', 'Buy': 'Buy', 'hold': 'Hold', 'Hold': 'Hold',
                      'sell': 'Sell', 'Sell': 'Sell', 'strongSell': 'Strong Sell',
                      'strong_sell': 'Strong Sell', 'Strong Sell': 'Strong Sell'}
            
            counts = {col_map[col]: latest[col] for col in rec.columns if col in col_map and pd.notna(latest[col]) and latest[col] > 0}
            
            if not counts:
                display(Markdown('**No valid recommendation data in recent period.**'))
            else:
                s = pd.Series(counts).reindex([p for p in ['Strong Buy', 'Buy', 'Hold', 'Sell', 'Strong Sell'] if p in counts])
                
                fig, ax = plt.subplots(figsize=(10, 6))
                
                color_map = {'Strong Buy': '#00aa00', 'Buy': '#66cc66', 'Hold': '#ffaa00', 'Sell': '#ff6666', 'Strong Sell': '#cc0000'}
                colors = [color_map.get(label, '#7f7f7f') for label in s.index]
                
                bars = ax.bar(s.index, s.values, color=colors, edgecolor='black', alpha=0.85, width=0.6)
                
                period = f" (Period: {latest['period']})" if 'period' in rec.columns else ""
                ax.set_title(f'Current Analyst Recommendations — {selected_stock}{period}', fontsize=14, fontweight='bold')
                ax.set_ylabel('Number of Analysts', fontsize=12)
                ax.set_xlabel('Recommendation', fontsize=12)
                plt.xticks(rotation=45, ha='right')
                ax.grid(axis='y', alpha=0.3, linestyle='--')
                
                for bar in bars:
                    if (h := bar.get_height()) > 0:
                        ax.annotate(f'{int(h)}', xy=(bar.get_x() + bar.get_width() / 2, h), xytext=(0, 3),
                                  textcoords='offset points', ha='center', va='bottom', fontsize=11, fontweight='bold')
                
                plt.tight_layout()
                plt.show()
        else:
            display(Markdown(f'**Unexpected type: {type(rec)}**'))
    except Exception as e:
        display(Markdown(f'**Error:** {e}'))
        import traceback
        print(traceback.format_exc())

[Back to Top](#table-of-contents)

## Market Intelligence

In [ ]:
# Fear and Greed Index Data
headers_list = [
    {"User-Agent": "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/120.0.0.0 Safari/537.36", "Accept": "application/json, text/javascript, */*; q=0.01", "Referer": "https://www.cnn.com/"},
    {"User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/118.0.0.0 Safari/537.36", "Accept": "application/json, text/javascript, */*; q=0.01", "Referer": "https://www.cnn.com/"},
    {"User-Agent": "curl/7.88.1", "Accept": "*/*"}
]

session = requests.Session()
session.mount("https://", HTTPAdapter(max_retries=Retry(total=2, backoff_factor=0.8, status_forcelist=[429, 500, 502, 503, 504])))

resp = None
for h in headers_list:
    try:
        resp = session.get("https://production.dataviz.cnn.io/index/fearandgreed/graphdata", headers=h, timeout=8)
        if resp.status_code == 200:
            break
    except requests.exceptions.RequestException:
        resp = None

if not resp:
    print("No response received")
else:
    def parse_ts(ts):
        if ts is None:
            return pd.NaT
        try:
            if isinstance(ts, (int, float)):
                t = float(ts)
                return pd.to_datetime(t, unit='ms' if t > 1e12 else 's', utc=True) if t > 1e9 else pd.NaT
            if isinstance(ts, str):
                return pd.to_datetime(ts)
        except:
            return pd.NaT
        return pd.NaT

    data = resp.json()
    rows = []
    for mk in ['fear_and_greed', 'fear_and_greed_historical', 'market_momentum_sp500', 'market_momentum_sp125', 'stock_price_strength', 'stock_price_breadth', 'put_call_options', 'market_volatility_vix', 'market_volatility_vix_50', 'junk_bond_demand', 'safe_haven_demand']:
        entry = data.get(mk) if isinstance(data, dict) else None
        score, rating, timestamp = None, None, None
        if isinstance(entry, dict):
            score, rating, timestamp = entry.get('score'), entry.get('rating'), entry.get('timestamp')
            if (score is None or rating is None or timestamp is None) and isinstance(entry.get('data'), list) and entry.get('data'):
                last = entry['data'][-1]
                if isinstance(last, dict):
                    score = score if score is not None else (last.get('y') or last.get('value'))
                    rating = rating if rating is not None else (last.get('rating') or last.get('label'))
                    timestamp = timestamp if timestamp is not None else last.get('x')
        rows.append({'metric': mk, 'score': score, 'rating': rating, 'timestamp': parse_ts(timestamp)})

    df = pd.DataFrame(rows).set_index('metric')
    df['score'] = pd.to_numeric(df['score'], errors='coerce')
    display(df)

In [ ]:
# Fear and greed index - historical trend plot
url = "https://production.dataviz.cnn.io/index/fearandgreed/graphdata"
headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/120.0.0.0 Safari/537.36",
    "Accept": "application/json, text/javascript, */*; q=0.01",
    "Referer": "https://www.cnn.com/",
}

session = requests.Session()
retries = Retry(total=2, backoff_factor=0.8, status_forcelist=[429, 500, 502, 503, 504])
session.mount("https://", HTTPAdapter(max_retries=retries))

try:
    resp = session.get(url, headers=headers, timeout=8)
    resp.raise_for_status()
    data = resp.json()
except Exception as e:
    raise SystemExit(f"Error fetching data: {e}")

# --- Parse data ---
def parse_timestamp(ts):
    """Convert CNN timestamp to pandas datetime"""
    return pd.to_datetime(ts, unit='ms', utc=True, errors='coerce')

historical = data.get("fear_and_greed_historical", {}).get("data", [])
if not historical:
    raise SystemExit("No historical data available.")

df = pd.DataFrame([
    {"timestamp": parse_timestamp(item.get("x")), "score": item.get("y")}
    for item in historical if item.get("x") and item.get("y") is not None
])
df = df.dropna().sort_values("timestamp")
df = df[(df["score"] >= 0) & (df["score"] <= 100)]

# --- Plot ---
def get_color(value):
    if value <= 25: return '#DC2626'   # Extreme Fear
    if value <= 45: return '#F97316'   # Fear
    if value <= 55: return '#FCD34D'   # Neutral
    if value <= 75: return '#10B981'   # Greed
    return '#059669'                   # Extreme Greed

def get_rating(value):
    if value <= 25: return 'Extreme Fear'
    if value <= 45: return 'Fear'
    if value <= 55: return 'Neutral'
    if value <= 75: return 'Greed'
    return 'Extreme Greed'

fig, ax = plt.subplots(figsize=(18, 6))

# Main line
ax.plot(df['timestamp'], df['score'], color='#1e3a8a', linewidth=3, alpha=0.9, zorder=3)

# Color fill
for i in range(len(df) - 1):
    x = [df['timestamp'].iloc[i], df['timestamp'].iloc[i + 1]]
    y = [df['score'].iloc[i], df['score'].iloc[i + 1]]
    ax.fill_between(x, 0, y, color=get_color(np.mean(y)), alpha=0.3)

# Reference lines
for y, color in [(25, '#DC2626'), (45, '#F97316'), (55, '#FCD34D'), (75, '#10B981')]:
    ax.axhline(y=y, color=color, linestyle='--', alpha=0.5, linewidth=1.5)

# Latest value annotation
last = df.iloc[-1]
ax.scatter(last['timestamp'], last['score'], color=get_color(last['score']),
           s=200, edgecolors='white', linewidth=3, zorder=5)
ax.annotate(f"{last['score']:.1f}\n{get_rating(last['score'])}",
            xy=(last['timestamp'], last['score']), xytext=(20, 20),
            textcoords='offset points', fontsize=10, fontweight='bold',
            bbox=dict(boxstyle='round,pad=0.7', facecolor=get_color(last['score']),
                      edgecolor='white', alpha=0.9), color='white')

# Labels & style
ax.set_title("CNN Fear & Greed Index - Historical Trend", fontsize=16, fontweight='bold')
ax.set_xlabel("Date", fontsize=11)
ax.set_ylabel("Score (0–100)", fontsize=11)
ax.set_ylim(0, 100)
ax.grid(alpha=0.25, linestyle=':')
ax.tick_params(axis='x', rotation=45)
plt.tight_layout()
plt.show()

In [ ]:
# Fetch share-price insights
resp = make_cnn_api_request(f"https://production.dataviz.cnn.io/insights/share_price/{selected_stock}")

def extract_plain_text(obj):
    if not isinstance(obj, (dict, list)):
        return None
    if isinstance(obj, dict):
        for key in ['list_summary', 'listSummary', 'listsummary']:
            if key in obj:
                val = obj[key]
                if isinstance(val, dict):
                    for pk in ('plain_text', 'plainText', 'plaintext'):
                        if pk in val and isinstance(val[pk], str):
                            return val[pk].strip()
                if isinstance(val, str):
                    return val.strip()
        for v in obj.values():
            if res := extract_plain_text(v):
                return res
    if isinstance(obj, list):
        for item in obj:
            if res := extract_plain_text(item):
                return res
    return None

try:
    if not resp:
        display(Markdown("**No response received — cannot fetch insights.**"))
    elif resp.status_code != 200:
        display(Markdown(f"**Status:** {resp.status_code}. Response:\n\n{resp.text[:1000]}"))
    else:
        try:
            data = resp.json()
            if plain := extract_plain_text(data):
                display(Markdown(plain))
            else:
                display(Markdown(f"**No list_summary.plain_text found — JSON (truncated):**\n\n{json.dumps(data, indent=2)[:4000]}"))
        except ValueError:
            display(Markdown(f"**Failed to decode JSON**\n\n{resp.text[:2000]}"))
except Exception as e:
    display(Markdown(f"**Error:** {e}"))

In [ ]:
# Real-time stock price streaming via WebSocket (BTC-USD example)
#!pip install yfinance nest_asyncio

# Import modules
import nest_asyncio
import asyncio

# Allow nested async loops (needed for Jupyter)
nest_asyncio.apply()

# Define a message handler
def message_handler(message):
    # Each message is a dict with price updates
    if "id" in message and message["id"] == "BTC-USD":
        price = message.get("price")
        if price:
            print(f"BTC-USD: ${price:.2f}")

# Create async function for WebSocket 
async def stream_btc():
    async with yf.AsyncWebSocket(verbose=False) as ws:
        await ws.subscribe(["BTC-USD"])
        print("Connected to Yahoo Finance WebSocket. Streaming BTC-USD live...")
        await ws.listen(message_handler)

# Run it
#await stream_btc()         # Uncomment this line to start streaming (will run indefinitely)

In [ ]:
# Import required libraries for world map
import sys, io, folium
from concurrent.futures import ThreadPoolExecutor, as_completed

# Define major stock indices with full country names
country_indices = {
    'United States of America': '^GSPC', 'United Kingdom': '^FTSE', 'Germany': '^GDAXI', 'France': '^FCHI',
    'Japan': '^N225', 'China': '000001.SS', 'India': '^BSESN', 'Brazil': '^BVSP', 'Canada': '^GSPTSE',
    'Australia': '^AXJO', 'South Korea': '^KS11', 'Mexico': '^MXX', 'South Africa': '^JN0U.JO',
    'Spain': '^IBEX', 'Italy': 'FTSEMIB.MI', 'Netherlands': '^AEX', 'Switzerland': '^SSMI',
    'Sweden': '^OMX', 'Hong Kong': '^HSI', 'Singapore': '^STI', 'Taiwan': '^TWII', 'Indonesia': '^JKSE',
    'Malaysia': '^KLSE', 'Thailand': '^SET.BK', 'Norway': '^OSEAX', 'Denmark': '^OMXC25',
    'Finland': '^OMXH25', 'Belgium': '^BFX', 'Austria': '^ATX', 'Turkey': 'XU100.IS',
    'Poland': 'WIG20.WA', 'Israel': '^TA125.TA', 'Argentina': '^MERV', 'Chile': '^IPSA', 'New Zealand': '^NZ50'
}

warnings.filterwarnings('ignore')

def fetch_single_stock(country, ticker):
    try:
        old_stderr, sys.stderr = sys.stderr, io.StringIO()
        hist = yf.Ticker(ticker).history(period='5d')
        sys.stderr = old_stderr
        if len(hist) >= 2:
            latest, prev = hist['Close'].iloc[-1], hist['Close'].iloc[-2]
            return {'country': country, 'ticker': ticker, 'daily_change': ((latest - prev) / prev) * 100, 'close': latest}
    except:
        sys.stderr = old_stderr
    return None

def get_daily_changes():
    with ThreadPoolExecutor(max_workers=10) as executor:
        futures = {executor.submit(fetch_single_stock, c, t): c for c, t in country_indices.items()}
        data = [f.result() for f in as_completed(futures) if f.result()]
    return pd.DataFrame(data)

print("Fetching stock market data...")
df = get_daily_changes()
print(f"Successfully fetched data for {len(df)} countries")

def get_color(chg):
    return '#8B0000' if chg <= -2 else '#FF0000' if chg <= -1 else '#FF8C00' if chg < 0 else '#FFFF00' if chg <= 1 else '#90EE90' if chg <= 2 else '#006400'

def create_world_map(df):
    m = folium.Map(location=[20, 0], zoom_start=2, tiles='CartoDB positron', 
                   zoom_control=False, scrollWheelZoom=False, doubleClickZoom=False, boxZoom=False, keyboard=False)
    
    world_geo = requests.get('https://raw.githubusercontent.com/python-visualization/folium/master/examples/data/world-countries.json').json()
    
    folium.Choropleth(geo_data=world_geo, name='Stock Index Daily Change', data=df, columns=['country', 'daily_change'],
                      key_on='feature.properties.name', fill_color='RdYlGn', fill_opacity=0.8, line_opacity=0.3,
                      legend_name='Daily Change (%)', nan_fill_color='lightgray', nan_fill_opacity=0.4).add_to(m)
    
    for _, row in df.iterrows():
        for feature in world_geo['features']:
            if feature['properties']['name'] == row['country']:
                popup = f"<div style='font-family: Arial; width: 200px;'><b style='font-size: 14px;'>{row['country']}</b><br><hr style='margin: 5px 0;'><b>Index:</b> {row['ticker']}<br><b>Daily Change:</b> <span style='color: {'green' if row['daily_change'] >= 0 else 'red'}; font-size: 16px; font-weight: bold;'>{row['daily_change']:+.2f}%</span><br><b>Close:</b> {row['close']:.2f}</div>"
                color = get_color(row['daily_change'])
                folium.GeoJson({'type': 'FeatureCollection', 'features': [feature]},
                              style_function=lambda x, c=color: {'fillColor': c, 'color': 'black', 'weight': 1, 'fillOpacity': 0.8},
                              popup=folium.Popup(popup, max_width=250), name=row['country']).add_to(m)
                break
    
    folium.LayerControl().add_to(m)
    return m

print("Creating world map...")
world_map = create_world_map(df)
world_map.save('stock_index_world_map.html')
print("Map saved as 'stock_index_world_map.html'")
world_map

## Options

[Back to Top](#table-of-contents)

In [ ]:
expirations = ticker.options
print(expirations)

expiry = expirations[0]  # Take the nearest expiry
option_chain = ticker.option_chain(expiry)

calls = option_chain.calls
puts = option_chain.puts

print(calls.head())
print(puts.head())

In [ ]:
put_call_ratio = puts['openInterest'].sum() / calls['openInterest'].sum()
print(f"Put/Call Open Interest Ratio: {put_call_ratio:.2f}")

In [ ]:
max_call_strike = calls.loc[calls['openInterest'].idxmax(), 'strike']
max_put_strike = puts.loc[puts['openInterest'].idxmax(), 'strike']
print(f"Max Call OI Strike: {max_call_strike}")
print(f"Max Put OI Strike: {max_put_strike}")

In [ ]:
atm_strike = calls.iloc[(calls['strike'] - ticker.history(period="1d")['Close'].iloc[-1]).abs().argsort()[:1]]['strike'].values[0]
atm_call = calls[calls['strike'] == atm_strike]['lastPrice'].values[0]
atm_put = puts[puts['strike'] == atm_strike]['lastPrice'].values[0]
expected_move = atm_call + atm_put
print(f"Approx. Expected Move by {expiry}: ±${expected_move:.2f}")

In [ ]:
iv_data = []
for exp in expirations[:5]:  # Check first few expirations
    oc = ticker.option_chain(exp)
    iv = oc.calls['impliedVolatility'].mean()
    iv_data.append({'expiry': exp, 'avg_IV': iv})

iv_df = pd.DataFrame(iv_data)
print(iv_df)

In [ ]:
expiry = ticker.options[0]  # nearest expiry
chain = ticker.option_chain(expiry)
calls, puts = chain.calls, chain.puts
spot = ticker.history(period="1d")['Close'].iloc[-1]

# Calculate median and mean strikes
call_median_strike = calls['strike'].median()
put_median_strike = puts['strike'].median()
call_mean_strike = (calls['strike'] * calls['openInterest']).sum() / calls['openInterest'].sum()
put_mean_strike = (puts['strike'] * puts['openInterest']).sum() / puts['openInterest'].sum()

# Create subplots
fig, axs = plt.subplots(3, 1, figsize=(12, 12))

# First subplot: Open Interest by Strike
axs[0].bar(calls['strike'], calls['openInterest'], width=1.0, label='Calls OI', color='green', alpha=0.6)
axs[0].bar(puts['strike'], puts['openInterest'], width=1.0, label='Puts OI', color='red', alpha=0.6)
axs[0].axvline(spot, color='blue', linestyle='--', label=f'Spot Price (${spot:.2f})', linewidth=2)
axs[0].axvline(call_median_strike, color='darkgreen', linestyle=':', label=f'Call Median (${call_median_strike:.2f})', linewidth=1.5, alpha=0.8)
axs[0].axvline(put_median_strike, color='darkred', linestyle=':', label=f'Put Median (${put_median_strike:.2f})', linewidth=1.5, alpha=0.8)
axs[0].axvline(call_mean_strike, color='lime', linestyle='-.', label=f'Call Mean (${call_mean_strike:.2f})', linewidth=1.5, alpha=0.8)
axs[0].axvline(put_mean_strike, color='orange', linestyle='-.', label=f'Put Mean (${put_mean_strike:.2f})', linewidth=1.5, alpha=0.8)
axs[0].set_title(f'Open Interest by Strike — {expiry}')
axs[0].set_xlabel('Strike Price')
axs[0].set_ylabel('Open Interest')
axs[0].legend()

# Second subplot: Put/Call Open Interest Ratio by Expiry
ratios = []
for exp in ticker.options[:6]:
    oc = ticker.option_chain(exp)
    pcr = oc.puts['openInterest'].sum() / oc.calls['openInterest'].sum()
    ratios.append({'expiry': exp, 'put_call_ratio': pcr})

df_ratios = pd.DataFrame(ratios)
axs[1].plot(df_ratios['expiry'], df_ratios['put_call_ratio'], marker='o')
axs[1].set_title('Put/Call Open Interest Ratio by Expiry')
axs[1].set_xlabel('Expiration Date')
axs[1].set_ylabel('Put/Call Ratio')
axs[1].grid(True)

# Third subplot: Implied Volatility Term Structure
term = []
for exp in ticker.options[:8]:
    oc = ticker.option_chain(exp)
    iv_mean = (oc.calls['impliedVolatility'].mean() + oc.puts['impliedVolatility'].mean()) / 2
    term.append({'expiry': exp, 'avg_IV': iv_mean})

df_term = pd.DataFrame(term)
axs[2].plot(df_term['expiry'], df_term['avg_IV'], marker='o', color='purple')
axs[2].set_title('Implied Volatility Term Structure')
axs[2].set_xlabel('Expiration Date')
axs[2].set_ylabel('Average Implied Volatility')
axs[2].grid(True)

plt.tight_layout()
plt.show()

In [ ]:
# --- Load data ---
expiry = ticker.options[0]  # nearest expiry
chain = ticker.option_chain(expiry)
calls, puts = chain.calls, chain.puts
spot = ticker.history(period="1d")['Close'].iloc[-1]

# --- Prepare data ---
strikes = np.array(sorted(set(calls['strike']).intersection(puts['strike'])))

# Select 10 strikes around the spot price
closest_idx = np.argmin(abs(strikes - spot))
start_idx = max(0, closest_idx - 4)
end_idx = min(len(strikes), closest_idx + 5)
strikes = strikes[start_idx:end_idx]

call_oi = calls.set_index('strike').reindex(strikes)['openInterest'].fillna(0)
put_oi  = puts.set_index('strike').reindex(strikes)['openInterest'].fillna(0)

# --- Plot settings ---
bar_width = 0.4
x = np.arange(len(strikes))

plt.figure(figsize=(12,6))
plt.bar(x - bar_width/2, call_oi, width=bar_width, color='green', alpha=0.7, label='Calls OI')
plt.bar(x + bar_width/2, put_oi,  width=bar_width, color='red', alpha=0.7, label='Puts OI')

# --- Add current stock price marker ---
closest_strike = strikes[np.argmin(abs(strikes - spot))]
plt.axvline(x[np.where(strikes == closest_strike)[0][0]], color='blue', linestyle='--', label=f'Spot Price (${spot:.2f})')

# --- Labels and style ---
plt.title(f"Open Interest by Strike — {expiry}")
plt.xlabel("Strike Price")
plt.ylabel("Open Interest")
plt.xticks(x, strikes, rotation=45)
plt.legend()
plt.grid(axis='y', alpha=0.3)
plt.tight_layout()
plt.show()

In [ ]:
# --- Load option chain ---
expiry = ticker.options[0]  # nearest expiry
chain = ticker.option_chain(expiry)
calls, puts = chain.calls, chain.puts

# --- Merge call and put open interest by strike ---
oi_table = pd.merge(
    calls[['strike', 'openInterest', 'volume', 'impliedVolatility']],
    puts[['strike', 'openInterest', 'volume', 'impliedVolatility']],
    on='strike',
    suffixes=('_call', '_put')
)

# --- Sort by strike ---
oi_table = oi_table.sort_values(by='strike').reset_index(drop=True)

# --- Format the columns for readability ---
oi_table.columns = [
    "Strike",
    "Call OI",
    "Call Volume",
    "Call IV",
    "Put OI",
    "Put Volume",
    "Put IV"
]

# --- Round and clean up ---
oi_table["Call IV"] = (oi_table["Call IV"] * 100).round(2)
oi_table["Put IV"] = (oi_table["Put IV"] * 100).round(2)

print(f"Open Interest Table — {expiry}")
display(oi_table.head(20))  # show first 20 rows